<a href="https://colab.research.google.com/github/bthodla/danano/blob/master/prj4/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data to be gathered

1.   Load the "twitter-archive-enhanced.csv" into a dataframe
2.   Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library (URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)
3. Using tweet ids from (1) above and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest
4. Consumer API keys: 
    
    Z22dLMPEOtCL2ayqnPTbFg3sK (API key)
    
    SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj (API secret key)
    
    Access token & access token secret

    14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb (Access token)

    nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313 (Access token secret)

    Read and write (Access level)




In [0]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import io
import tweepy
import json
from google.colab import drive
import yaml
import time
import os


In [0]:
def load_yaml_file(filename: str) -> dict:
    twitter_keys = {}

    with open(filename, 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    
    twitter_keys['consumer_key'] = cfg['twitter']['consumer']['key']
    twitter_keys['consumer_secret'] = cfg['twitter']['consumer']['secret']
    twitter_keys['access_token'] = cfg['twitter']['access']['token']
    twitter_keys['access_token_secret'] = cfg['twitter']['access']['token-secret']

    return twitter_keys

In [0]:
def print_file(file_name, lines = 10):
    with open(file_name, 'r') as f:
        for _ in range(lines):
            print (f.readline())
        

In [79]:
drive.mount('/content/gdrive')
twitter_keys_yaml_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_keys.yaml'
tweet_json_file = '/content/gdrive/My Drive/Colab Notebooks/tweet_json.txt'

"""
with open(twitter_keys_yaml_file, encoding='utf-8') as file:
    line = file.readline()
    while line:
        line = file.readline()
        print(line)
"""
print(load_yaml_file(twitter_keys_yaml_file))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'consumer_key': 'Z22dLMPEOtCL2ayqnPTbFg3sK', 'consumer_secret': 'SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj', 'access_token': '14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb', 'access_token_secret': 'nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313'}


In [80]:
# 1. Load the file "twitter-archive-enhanced.csv" into a dataframe
twitter_archive_file_url = 'https://raw.githubusercontent.com/bthodla/danano/master/prj4/twitter-archive-enhanced.csv'

tweets_df = pd.read_csv(twitter_archive_file_url)

tweet_count = tweets_df.shape[0]
print ('%s %d' % ('Tweet Count: ', tweet_count))

Tweet Count:  2356


In [81]:
# 2. Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library

image_predictions_file_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

url_data = requests.get(image_predictions_file_url).content

img_pred_df = pd.read_csv(io.StringIO(url_data.decode('utf-8')), sep = '\t')

img_pred_df.shape

(2075, 12)

In [82]:
""" 3. Using tweet ids from tweet_df and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items 
and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include 
tweet id, retweet count, favorite count and any other data items of interest
"""

twitter_keys = load_yaml_file(twitter_keys_yaml_file)
consumer_key = twitter_keys['consumer_key']
consumer_secret = twitter_keys['consumer_secret']
access_token = twitter_keys['access_token']
access_token_secret = twitter_keys['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

error_tweets = {}
tweet_offset = 0
tweet_batch = 100
tweets_remaining = 0

# Delete the file first so that each run can add content fresh
if os.path.isfile(tweet_json_file):
    os.remove(tweet_json_file)

with open(tweet_json_file, mode = 'a', encoding = 'utf-8') as f:
    while tweet_batch:
        tweet_ids = tweets_df[tweet_offset:tweet_offset + tweet_batch]['tweet_id']
        for tweet_id in tweet_ids:
            try:
                status = api.get_status(tweet_id)
                f.write(json.dumps(status._json) + '\n')

            except tweepy.TweepError as e:
                error_tweets[status.id] = e
            
        tweets_remaining = tweet_count - tweet_offset
        tweet_batch = tweet_batch if tweet_batch < tweets_remaining else tweets_remaining
        tweet_offset += tweet_batch
        # print ('%s %d %s %d' % ('Tweets Remaining: ', tweets_remaining, 'Tweet Offset: ', tweet_offset))
        # time.sleep(1 * 5)


Rate limit reached. Sleeping for: 770
Rate limit reached. Sleeping for: 769


In [83]:
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    print(len(f.readlines()))

2339


In [86]:
print_file (tweet_json_file)

{"created_at": "Tue Aug 01 16:23:56 +0000 2017", "id": 892420643555336193, "id_str": "892420643555336193", "text": "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU", "truncated": false, "entities": {"hashtags": [], "symbols": [], "user_mentions": [], "urls": [], "media": [{"id": 892420639486877696, "id_str": "892420639486877696", "indices": [86, 109], "media_url": "http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg", "media_url_https": "https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg", "url": "https://t.co/MgUWQ76dJU", "display_url": "pic.twitter.com/MgUWQ76dJU", "expanded_url": "https://twitter.com/dog_rates/status/892420643555336193/photo/1", "type": "photo", "sizes": {"thumb": {"w": 150, "h": 150, "resize": "crop"}, "medium": {"w": 540, "h": 528, "resize": "fit"}, "small": {"w": 540, "h": 528, "resize": "fit"}, "large": {"w": 540, "h": 528, "resize": "fit"}}}]}, "extended_entities": {"media": [{"id": 892420639486877696, "id_s